In [11]:
import os
os.environ['KAGGLE_USERNAME'] = 'malakyousof'
os.environ['KAGGLE_KEY'] = '14a6c5c64d5fd62e0b83d76777c55b9b'


In [ ]:
!pip install mxnet==1.8.0.post0
!pip install "pandas>=1.2.4,<1.3.0"
!pip install autogluon


In [4]:

import pandas as pd
import autogluon
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print("Train shape:", train.shape)
print(train.info())
print(train.describe())a
print("Missing values:\n", train.isnull().sum())
print(train.columns)

Train shape: (10886, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB
None
             season       holiday    workingday       weather         temp  \
count  10886.000000  10886.000000  10886.000000  10886.000000  10886.00000   
mean       2.506614      0.028569      0.680875      1.418427     

In [5]:
import pandas as pd

# Assuming train and test are already loaded DataFrames

# Extract hour from datetime
train['hour'] = pd.to_datetime(train['datetime']).dt.hour
test['hour'] = pd.to_datetime(test['datetime']).dt.hour

# Create a new ratio feature
train['temp_atemp_ratio'] = train['temp'] / (train['atemp'] + 1e-5)
test['temp_atemp_ratio'] = test['temp'] / (test['atemp'] + 1e-5)

# Drop leakage columns only if they exist
cols_to_drop = ['casual', 'registered']
cols_present = [col for col in cols_to_drop if col in train.columns]
train = train.drop(columns=cols_present)

# Save updated DataFrames
train.to_csv('train_modified.csv', index=False)
test.to_csv('test_modified.csv', index=False)


In [6]:
label = 'count'

train_features = train.drop(columns=[label, 'datetime']) 
train_label = train[label]

test_features = test.drop(columns=['datetime'])  


In [ ]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# Load your processed train and test data
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')



predictor = TabularPredictor(label=label).fit(
    train_data=train,
    tuning_data=None,  
    time_limit=600, 
    presets='best_quality',  
    hyperparameters={
        'GBM': {
            'num_boost_round': 1000,
            'learning_rate': 0.05,
            'max_depth': 10,
        },
        # You can add other models too
    },
    verbosity=3
)


In [8]:
predictions = predictor.predict(test)


/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)
Loading: AutogluonModels/ag-20250608_121543/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20250608_121543/models/WeightedEnsemble_L2/model.pkl


In [20]:
!pip install kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 6.0 MB/s eta 0:00:00


In [22]:
import os

competition_name = 'bike-sharing-demand'  
submission_file = 'malakmodel.csv'
message = 'AutoGluon model submission'

os.system(f'kaggle competitions submit -c {competition_name} -f {submission_file} -m "{message}"')



100%|██████████| 188k/188k [00:00<00:00, 405kB/s]


Successfully submitted to Bike Sharing Demand

0

/usr/bin/sh: 1: cannot open competition-name: No such file
/usr/bin/sh: 1: kaggle: not found
